In [3]:
!pip install langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.4 MB/s eta 0:00:00


In [4]:
from langgraph.graph import StateGraph, END
#from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
#from langchain.utilities import PythonREPL
from langchain_experimental.utilities import PythonREPL

In [ ]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1-mini",         # deployment name from Azure portal
    azure_endpoint="https://singhlkaay-7416-resource.cognitiveservices.azure.com/",           # your endpoint
    api_version="2023-12-01-preview",
    api_key="mykey",
    temperature=0.0
)

In [6]:
# Shared state
class State(dict): pass

In [7]:
# Nodes
def classify_problem(state: State):
    """Optionally LLM could be used to decide whether math is small or large."""
    expr = state["query"]
    # crude heuristic: look for big exponents
    if "**" in expr and int(expr.split("**")[1].split()[0]) > 20:
        state["route"] = "tool"
    else:
        state["route"] = "direct"
    return state

def direct_compute(state: State):
    """Handle small computations directly."""
    # Example: just hardcode simple eval here
    state["answer"] = eval(state["query"].replace("log", "(__import__('math').log)"))
    return state

def python_tool_compute(state: State):
    """Use Python REPL for large computations."""
    repl = PythonREPL()
    result = repl.run(state["query"].replace("log", "(__import__('math').log)"))
    state["answer"] = result
    return state

In [8]:
# Build graph
graph = StateGraph(State)
graph.add_node("classify", classify_problem)
graph.add_node("direct", direct_compute)
graph.add_node("tool", python_tool_compute)
#graph.add_node("extra",myown_func)

graph.set_entry_point("classify")
graph.add_conditional_edges("classify", lambda s: s["route"], {"direct": "direct", "tool": "tool"})
graph.add_edge("direct", END)
graph.add_edge("tool", END)

In [21]:
#query = {"query": "10000 * 12**100 - 12 + log(10**1000)"}

In [22]:
# Run
app = graph.compile()
#state = app.invoke(query["query"])
#print("Final Answer:", state["answer"])

In [ ]:
#Using LLM and validation with user input
def classify_with_llm(state: State):
    """Use LLM to decide whether problem is SMALL or LARGE."""
    expr = state["query"]
    prompt = f"""
    You are a reasoning assistant.
    Task: Decide if this math problem should be solved directly or with a calculator.
    If it has large exponents (like **50 or higher), say "LARGE".
    Otherwise, say "SMALL".

    Problem: {expr}
    Answer only with SMALL or LARGE.
    """
    decision = llm.invoke(prompt).content.strip()
    state["llm_decision"] = decision
    return state

def user_validation(state: State):
    """Ask user to validate the LLM's choice before execution."""
    decision = state["llm_decision"]
    print(f"🤖 LLM thinks this problem is: {decision}")
    user_input = input("Do you agree? (yes/no): ").strip().lower()

    if user_input == "no":
        # Let user override
        override = input("Enter your choice (SMALL/LARGE): ").strip().upper()
        state["route"] = override
    else:
        state["route"] = decision
    return state

def direct_compute(state: State):
    """Handle small computations directly."""
    state["answer"] = eval(state["query"].replace("log", "(__import__('math').log)"))
    return state

def python_tool_compute(state: State):
    """Use Python REPL for large computations."""
    repl = PythonREPL()
    result = repl.run(state["query"].replace("log", "(__import__('math').log)"))
    state["answer"] = result
    return state

# --- Build the graph ---
graph = StateGraph(State)

graph.add_node("classify", classify_with_llm)
graph.add_node("validate", user_validation)
graph.add_node("direct", direct_compute)
graph.add_node("tool", python_tool_compute)

graph.set_entry_point("classify")
graph.add_edge("classify", "validate")

# Conditional edge after validation
graph.add_conditional_edges(
    "validate",
    lambda s: s["route"],
    {"SMALL": "direct", "LARGE": "tool"}
)

graph.add_edge("direct", END)
graph.add_edge("tool", END)

# --- Run it ---
app = graph.compile()

state = app.invoke({"query": "10000 * 12**100 - 12 + log(10**1000)"})
print("Final Answer:", state["answer"])

In [ ]:
#implementing agents, with langraph and call to llm, user validation of llm response
# Tool: Python calculator
python_tool = Tool(
    name="Python Calculator",
    func=PythonREPL().run,
    description="Execute Python for large math computations."
)

# --- Agents ---
# Agent 1: Classifier Agent
classifier_prompt = """
You are a classification agent.
Task: Look at the given math expression and decide if it's SMALL or LARGE.
Rule: If exponent >= 50, it's LARGE. Otherwise, it's SMALL.
Answer ONLY 'SMALL' or 'LARGE'.
"""
classifier_agent = initialize_agent([], llm, agent="zero-shot-react-description", verbose=True)

def classify_with_agent(state: State):
    expr = state["query"]
    response = classifier_agent.run(classifier_prompt + f"\nExpression: {expr}")
    state["llm_decision"] = response.strip().upper()
    return state

# User validation node
def user_validation(state: State):
    decision = state["llm_decision"]
    print(f" Classifier Agent thinks this problem is: {decision}")
    user_input = input("Do you agree? (yes/no): ").strip().lower()

    if user_input == "no":
        override = input("Enter your choice (SMALL/LARGE): ").strip().upper()
        state["route"] = override
    else:
        state["route"] = decision
    return state

# Agent 2: Math Agent (can use tool)
math_agent = initialize_agent([python_tool], llm, agent="zero-shot-react-description", verbose=True)

def solve_with_agent(state: State):
    expr = state["query"]

    if state["route"] == "SMALL":
        # Direct computation (no tool)
        state["answer"] = eval(expr.replace("log", "(__import__('math').log)"))
    else:
        # Delegate to math agent with tool
        result = math_agent.run(expr.replace("log", "(__import__('math').log)"))
        state["answer"] = result

    return state

# --- Build LangGraph workflow ---
graph = StateGraph(State)

graph.add_node("classify", classify_with_agent)
graph.add_node("validate", user_validation)
graph.add_node("solve", solve_with_agent)

graph.set_entry_point("classify")
graph.add_edge("classify", "validate")
graph.add_edge("validate", "solve")
graph.add_edge("solve", END)

# --- Run ---
app = graph.compile()

state = app.invoke({"query": "10000 * 12**100 - 12 + log(10**1000)"})
print("Final Answer:", state["answer"])